# Statistical Correlation Exploration between Sport Teams' Win/Loss Ratio and Population of corresponding Regions
## Description
In this assignment I read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answered some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). 

All questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data I'm working with, which reuired some googling.

For each sport I answered the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so we are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Averaged the win/loss ratios for those cities which have multiple teams of a single sport. I only used data **from year 2018** for your analysis -- this is important!


## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [4]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re


In [5]:

def nhl_correlation(): 
    
    nhl_df=pd.read_csv("assets/nhl.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    
    cities['NHL'] = cities['NHL'].replace(to_replace='\[.*\]', value='',regex =True)
    
    cities = cities.iloc[:,[0,1,5]]
    
    nhl_df[nhl_df['year']==2018]
    
    nhl_df = nhl_df[nhl_df['year']==2018]
    nhl_df = nhl_df.drop([0,9,18,26],axis=0)
    
    nhl_df = nhl_df.loc[:,['team','W','L']].reset_index()
    del nhl_df['index']
    
    nhl_df['W'] = [int(x) for x in nhl_df['W']]
    nhl_df['L'] = [int(x) for x in nhl_df['L']]
    nhl_df['W/L Ratio'] = nhl_df['W']/(nhl_df['W'] + nhl_df['L'])
    
    nhl_df['team'] = nhl_df['team'].replace(to_replace='\*', value='',regex=True)
    cities['NHL'] = cities['NHL'].replace(to_replace=['','—','  ',None],value='0')
    
    diction = {
    'Florida Panthers':'Miami–Fort Lauderdale',
     'New Jersey Devils':'New York City',
     'Carolina Hurricanes':'Raleigh',
     'New York Islanders':'New York City',
     'New York Rangers':'New York City',
     'Minnesota Wild':'Minneapolis–Saint Paul',
     'Colorado Avalanche':'Denver',
     'Dallas Stars':'Dallas–Fort Worth',
     'Vegas Golden Knights':'Las Vegas',
     'Anaheim Ducks':'Los Angeles',
     'Los Angeles Kings':'Los Angeles',
     'Arizona Coyotes':'Phoenix'}

    teamcity = {}

    for i in cities.iterrows():
    
        for j in nhl_df['team']:
            s = i[1]['Metropolitan area'].split()[0].strip(" ,.-_")
            so = i[1]['NHL']
            if (re.search(s,j)) and (re.search(so,j)):
                teamcity[j] = i[1]['Metropolitan area']
            
    teamcity.update(diction)
    nhl_df['city'] = nhl_df['team'].map(teamcity)
    
    new = pd.merge(nhl_df,cities, left_on= "city", right_on= "Metropolitan area")
    new = new[['team','W/L Ratio','city','Population (2016 est.)[8]']]
    new = new.groupby('city').agg({'W/L Ratio': np.average, 'Population (2016 est.)[8]': lambda x: [int(i) for i in x][0]})
   
    
    population_by_region = new['Population (2016 est.)[8]'] 
    win_loss_by_region = new['W/L Ratio'] 

    
    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

In [6]:
nhl_correlation()

0.012486162921209923

## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [7]:
def nba_correlation():
    nba = pd.read_csv("assets/nba.csv")
    cities = pd.read_html("assets/wikipedia_data.html")[1]
    cities = cities.iloc[:-1,[0,3,5,6,7,8]]

    cities = cities.iloc[:,[0,1,4]]

    cities['NBA'] = cities['NBA'].replace(to_replace='\[.*\]', value='',regex =True)
    
    nba = nba[nba['year']==2018]
    nba = nba.loc[:,['team','W','L','W/L%']]
    
    nba['team'] = nba['team'].replace(to_replace='\*|\*\(.*\)|\(.*\)', value='',regex=True)
    nba['team'] = nba['team'].replace(to_replace='Â', value='',regex=True)
    
    nba['W'] = [int(x) for x in nba['W']]
    nba['L'] = [int(x) for x in nba['L']]
    nba['W/L%'] = nba['W']/(nba['W'] + nba['L'])
    
    cities['NBA'] = cities['NBA'].replace(to_replace=['','—','  ',None],value='0')
   
    teamcity = {}

    for i in cities.iterrows():
    
        for j in nba['team']:
        
            s = i[1]['Metropolitan area'].split()[0].strip(" ,.-_")
            s1 = i[1]['Metropolitan area'].split("–")[0].strip(" ,.-_")
            so = i[1]['NBA']
            
            if (re.search(s,j)) and (re.search(so,j)):
                teamcity.update({j: i[1]['Metropolitan area']})
            
            elif (re.search(s1,j)) and (re.search(so,j)):
                teamcity.update({j: i[1]['Metropolitan area']})
                
    nba['city'] = nba['team'].map(teamcity)
    
    nba.iloc[4,4] = 'Indianapolis'
    nba.iloc[10,4] = 'New York City'
    nba.iloc[11,4] = 'New York City'

    nba.iloc[16,4] = 'San Francisco Bay Area'
    nba.iloc[19,4] = 'Salt Lake City'
    nba.iloc[22,4] = 'Minneapolis–Saint Paul'
    nba.iloc[24,4] = 'Los Angeles'
    nba.iloc[25,4] = 'Los Angeles'
    
    new = pd.merge(nba,cities, left_on= "city", right_on= "Metropolitan area")
    new = new[['team','W/L%','city','Population (2016 est.)[8]']]

    new = new.groupby('city').agg({'W/L%': np.average, 'Population (2016 est.)[8]': lambda x: [int(i) for i in x][0]})

    
    population_by_region = new['Population (2016 est.)[8]'] 
    win_loss_by_region = new['W/L%'] 


    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

In [8]:
nba_correlation()

-0.17657160252844623

## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [9]:
def mlb_correlation(): 
    
    mlb = pd.read_csv("assets/mlb.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]

    
    cities['MLB'] = cities['MLB'].replace(to_replace='\[.*\]', value='',regex =True)
    cities = cities.iloc[:,[0,1,3]]
    
    mlb = mlb[mlb['year']==2018]
    mlb = mlb[['team','W-L%']]
    
    cities['MLB'] = cities['MLB'].replace(to_replace=['','—','  ',None],value='0')
    
    teamcity = {}

    for i in cities.iterrows():
    
        for j in mlb['team']:
        
            so = i[1]['MLB']
        
            p1 = i[1]['Metropolitan area'].split()[0].strip(" ,.-_")
        
            s1 = i[1]['Metropolitan area'].split("–")[0].strip(" ,.-_")
        
            if len(i[1]['Metropolitan area'].split())>1:
            
                p2 = i[1]['Metropolitan area'].split()[1].strip(" ,.-_")
            
                if (re.search(p1,j)) and (re.search(p2,j)) and (re.search(so,j)):
                    teamcity.update({j: i[1]['Metropolitan area']})

            
            if (re.search(p1,j)) and (re.search(so,j)):
                teamcity.update({j: i[1]['Metropolitan area']})
            
            elif (re.search(s1,j)) and (re.search(so,j)):
                teamcity.update({j: i[1]['Metropolitan area']})
                         
    
    diction = {'New York Yankees':'New York City',
         'Minnesota Twins':'Minneapolis–Saint Paul', 
         'Chicago White Sox':'Chicago',
         'Oakland Athletics':'San Francisco Bay Area',
         'Los Angeles Angels':'Los Angeles',
         'Texas Rangers':'Dallas–Fort Worth',
         'New York Mets':'New York City',
         'Chicago Cubs':'Chicago',
         'Los Angeles Dodgers':'Los Angeles',
         'Colorado Rockies':'Denver',
         'Arizona Diamondbacks':'Phoenix',
         'San Francisco Giants':'San Francisco Bay Area'}
    
    teamcity.update(diction)
    mlb['city'] = mlb['team'].map(teamcity)
    
    new = pd.merge(mlb,cities, left_on= "city", right_on= "Metropolitan area")
    new = new[['team','W-L%','city','Population (2016 est.)[8]']]
    
    new = new.groupby('city').agg({'W-L%': np.average, 'Population (2016 est.)[8]': lambda x: [int(i) for i in x][0]})
    
    
    population_by_region = new['Population (2016 est.)[8]']
    win_loss_by_region = new['W-L%']

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

In [10]:
mlb_correlation()

0.15003737475409498

## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [13]:
def nfl_correlation(): 
    nfl = pd.read_csv("assets/nfl.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]


    cities['NFL'] = cities['NFL'].replace(to_replace='\[.*\]', value='',regex =True)
    cities = cities.iloc[:,[0,1,2]]
    
    nfl = nfl[nfl['year']==2018]
    nfl = nfl[['team','W-L%']].drop([0,5,10,15,20,25,30,35], axis=0).reset_index()
    
    cities['NFL'] = cities['NFL'].replace(to_replace=['','—','  ',None],value='0')
    del nfl['index']
    
    nfl['team'] = nfl['team'].replace(to_replace='\*|\+', value='',regex=True)
    nfl['W-L%'] = [float(x) for x in nfl['W-L%']]
    
    teamcity = {}

    for i in cities.iterrows():
    
        for j in nfl['team']:
            s = i[1]['Metropolitan area'].split()[0].strip(" ,.-_")
            s1 = i[1]['Metropolitan area'].split("–")[0].strip(" ,.-_")
            so = i[1]['NFL']
        
            if (re.search(s,j)) and (re.search(so,j)):
                teamcity.update({j: i[1]['Metropolitan area']})
            elif (re.search(s1,j)) and (re.search(so,j)):
                teamcity.update({j: i[1]['Metropolitan area']})
                
    diction = {'New England Patriots':'Boston',
     'New York Jets':'New York City',
     'Tennessee Titans':'Nashville',
     'Los Angeles Chargers':'Los Angeles',
     'Oakland Raiders':'San Francisco Bay Area',
     'New York Giants':'New York City',
     'Minnesota Vikings':'Minneapolis–Saint Paul',
     'Carolina Panthers':'Charlotte',
     'Los Angeles Rams':'Los Angeles',
     'San Francisco 49ers':'San Francisco Bay Area',
     'Arizona Cardinals':'Phoenix'}
    
    teamcity.update(diction)
    nfl['city'] = nfl['team'].map(teamcity)
    
    new = pd.merge(nfl,cities, left_on= "city", right_on= "Metropolitan area")
    new = new[['team','W-L%','city','Population (2016 est.)[8]']]
    
    new = new.groupby('city').agg({'W-L%': np.average, 'Population (2016 est.)[8]': lambda x: [int(i) for i in x][0]})


    population_by_region = new['Population (2016 est.)[8]']
    win_loss_by_region = new['W-L%']


    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

In [14]:
nfl_correlation()

0.004282141436393022

## Question 5
In this question explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate.

In [15]:
def nhl_correlation(): 
 
    nhl_df = pd.read_csv("assets/nhl.csv")

    cities = pd.read_html("assets/wikipedia_data.html")[1]
    cities = cities.iloc[:-1,[0,3,5,6,7,8]]
    
    cities['NHL'] = cities['NHL'].replace(to_replace='\[.*\]', value='',regex =True)
    
    cities = cities.iloc[:,[0,1,5]]
    
    nhl_df[nhl_df['year']==2018]
    
    nhl_df = nhl_df[nhl_df['year']==2018]
    nhl_df = nhl_df.drop([0,9,18,26],axis=0)
    
    nhl_df = nhl_df.loc[:,['team','W','L']].reset_index()
    del nhl_df['index']
    
    nhl_df['W'] = [int(x) for x in nhl_df['W']]
    nhl_df['L'] = [int(x) for x in nhl_df['L']]
    nhl_df['W/L Ratio'] = nhl_df['W']/(nhl_df['W'] + nhl_df['L'])
    
    nhl_df['team'] = nhl_df['team'].replace(to_replace='\*', value='',regex=True)
    cities['NHL'] = cities['NHL'].replace(to_replace=['','—','  ',None],value='0')
    
    diction = {
    'Florida Panthers':'Miami–Fort Lauderdale',
     'New Jersey Devils':'New York City',
     'Carolina Hurricanes':'Raleigh',
     'New York Islanders':'New York City',
     'New York Rangers':'New York City',
     'Minnesota Wild':'Minneapolis–Saint Paul',
     'Colorado Avalanche':'Denver',
     'Dallas Stars':'Dallas–Fort Worth',
     'Vegas Golden Knights':'Las Vegas',
     'Anaheim Ducks':'Los Angeles',
     'Los Angeles Kings':'Los Angeles',
     'Arizona Coyotes':'Phoenix'}

    teamcity = {}

    for i in cities.iterrows():
    
        for j in nhl_df['team']:
            s = i[1]['Metropolitan area'].split()[0].strip(" ,.-_")
            so = i[1]['NHL']
            if (re.search(s,j)) and (re.search(so,j)):
                teamcity[j] = i[1]['Metropolitan area']
            
    teamcity.update(diction)
    nhl_df['city'] = nhl_df['team'].map(teamcity)
    
    new = pd.merge(nhl_df,cities, left_on= "city", right_on= "Metropolitan area")
    new = new[['team','W/L Ratio','city','Population (2016 est.)[8]']]
    new = new.groupby('city').agg({'W/L Ratio': np.average, 'Population (2016 est.)[8]': lambda x: [int(i) for i in x][0]})
   
    return new

def nba_correlation():
   
    nba = pd.read_csv("assets/nba.csv")


    cities = pd.read_html("assets/wikipedia_data.html")[1]
    cities = cities.iloc[:-1,[0,3,5,6,7,8]]
    cities = cities.iloc[:,[0,1,4]]

    cities['NBA'] = cities['NBA'].replace(to_replace='\[.*\]', value='',regex =True)
    
    nba = nba[nba['year']==2018]
    nba = nba.loc[:,['team','W','L','W/L%']]
    
    nba['team'] = nba['team'].replace(to_replace='\*|\*\(.*\)|\(.*\)', value='',regex=True)
    nba['team'] = nba['team'].replace(to_replace='Â', value='',regex=True)
    
    nba['W'] = [int(x) for x in nba['W']]
    nba['L'] = [int(x) for x in nba['L']]
    nba['W/L%'] = nba['W']/(nba['W'] + nba['L'])
    
    cities['NBA'] = cities['NBA'].replace(to_replace=['','—','  ',None],value='0')
   
    teamcity = {}

    for i in cities.iterrows():
    
        for j in nba['team']:
        
            s = i[1]['Metropolitan area'].split()[0].strip(" ,.-_")
            s1 = i[1]['Metropolitan area'].split("–")[0].strip(" ,.-_")
            so = i[1]['NBA']
            
            if (re.search(s,j)) and (re.search(so,j)):
                teamcity.update({j: i[1]['Metropolitan area']})
            
            elif (re.search(s1,j)) and (re.search(so,j)):
                teamcity.update({j: i[1]['Metropolitan area']})
                
    nba['city'] = nba['team'].map(teamcity)
    
    nba.iloc[4,4] = 'Indianapolis'
    nba.iloc[10,4] = 'New York City'
    nba.iloc[11,4] = 'New York City'

    nba.iloc[16,4] = 'San Francisco Bay Area'
    nba.iloc[19,4] = 'Salt Lake City'
    nba.iloc[22,4] = 'Minneapolis–Saint Paul'
    nba.iloc[24,4] = 'Los Angeles'
    nba.iloc[25,4] = 'Los Angeles'
    
    new = pd.merge(nba,cities, left_on= "city", right_on= "Metropolitan area")
    new = new[['team','W/L%','city','Population (2016 est.)[8]']]

    new = new.groupby('city').agg({'W/L%': np.average, 'Population (2016 est.)[8]': lambda x: [int(i) for i in x][0]})

    return new

def mlb_correlation(): 
    mlb = pd.read_csv("assets/mlb.csv")

    cities = pd.read_html("assets/wikipedia_data.html")[1]
    cities = cities.iloc[:-1,[0,3,5,6,7,8]]
    
    cities['MLB'] = cities['MLB'].replace(to_replace='\[.*\]', value='',regex =True)
    cities = cities.iloc[:,[0,1,3]]
    
    mlb = mlb[mlb['year']==2018]
    mlb = mlb[['team','W-L%']]
    
    cities['MLB'] = cities['MLB'].replace(to_replace=['','—','  ',None],value='0')
    
    teamcity = {}

    for i in cities.iterrows():
    
        for j in mlb['team']:
        
            so = i[1]['MLB']
        
            p1 = i[1]['Metropolitan area'].split()[0].strip(" ,.-_")
        
            s1 = i[1]['Metropolitan area'].split("–")[0].strip(" ,.-_")
        
            if len(i[1]['Metropolitan area'].split())>1:
            
                p2 = i[1]['Metropolitan area'].split()[1].strip(" ,.-_")
            
                if (re.search(p1,j)) and (re.search(p2,j)) and (re.search(so,j)):
                    teamcity.update({j: i[1]['Metropolitan area']})

            
            if (re.search(p1,j)) and (re.search(so,j)):
                teamcity.update({j: i[1]['Metropolitan area']})
            
            elif (re.search(s1,j)) and (re.search(so,j)):
                teamcity.update({j: i[1]['Metropolitan area']})
                         
    
    diction = {'New York Yankees':'New York City',
         'Minnesota Twins':'Minneapolis–Saint Paul', 
         'Chicago White Sox':'Chicago',
         'Oakland Athletics':'San Francisco Bay Area',
         'Los Angeles Angels':'Los Angeles',
         'Texas Rangers':'Dallas–Fort Worth',
         'New York Mets':'New York City',
         'Chicago Cubs':'Chicago',
         'Los Angeles Dodgers':'Los Angeles',
         'Colorado Rockies':'Denver',
         'Arizona Diamondbacks':'Phoenix',
         'San Francisco Giants':'San Francisco Bay Area'}
    
    teamcity.update(diction)
    mlb['city'] = mlb['team'].map(teamcity)
    
    new = pd.merge(mlb,cities, left_on= "city", right_on= "Metropolitan area")
    new = new[['team','W-L%','city','Population (2016 est.)[8]']]
    
    new = new.groupby('city').agg({'W-L%': np.average, 'Population (2016 est.)[8]': lambda x: [int(i) for i in x][0]})
    return new
    
def nfl_correlation(): 

    nfl = pd.read_csv("assets/nfl.csv")

    cities = pd.read_html("assets/wikipedia_data.html")[1]
    cities = cities.iloc[:-1,[0,3,5,6,7,8]]

    cities['NFL'] = cities['NFL'].replace(to_replace='\[.*\]', value='',regex =True)
    cities = cities.iloc[:,[0,1,2]]
    
    nfl = nfl[nfl['year']==2018]
    nfl = nfl[['team','W-L%']].drop([0,5,10,15,20,25,30,35], axis=0).reset_index()
    
    cities['NFL'] = cities['NFL'].replace(to_replace=['','—','  ',None],value='0')
    del nfl['index']
    
    nfl['team'] = nfl['team'].replace(to_replace='\*|\+', value='',regex=True)
    nfl['W-L%'] = [float(x) for x in nfl['W-L%']]
    
    teamcity = {}

    for i in cities.iterrows():
    
        for j in nfl['team']:
            s = i[1]['Metropolitan area'].split()[0].strip(" ,.-_")
            s1 = i[1]['Metropolitan area'].split("–")[0].strip(" ,.-_")
            so = i[1]['NFL']
        
            if (re.search(s,j)) and (re.search(so,j)):
                teamcity.update({j: i[1]['Metropolitan area']})
            elif (re.search(s1,j)) and (re.search(so,j)):
                teamcity.update({j: i[1]['Metropolitan area']})
                
    diction = {'New England Patriots':'Boston',
     'New York Jets':'New York City',
     'Tennessee Titans':'Nashville',
     'Los Angeles Chargers':'Los Angeles',
     'Oakland Raiders':'San Francisco Bay Area',
     'New York Giants':'New York City',
     'Minnesota Vikings':'Minneapolis–Saint Paul',
     'Carolina Panthers':'Charlotte',
     'Los Angeles Rams':'Los Angeles',
     'San Francisco 49ers':'San Francisco Bay Area',
     'Arizona Cardinals':'Phoenix'}
    
    teamcity.update(diction)
    nfl['city'] = nfl['team'].map(teamcity)
    
    new = pd.merge(nfl,cities, left_on= "city", right_on= "Metropolitan area")
    new = new[['team','W-L%','city','Population (2016 est.)[8]']]
    
    new = new.groupby('city').agg({'W-L%': np.average, 'Population (2016 est.)[8]': lambda x: [int(i) for i in x][0]})

    return new
    
    
def sports_team_performance():
    
    nfl = nfl_correlation()
    nba = nba_correlation()
    mlb = mlb_correlation()
    nhl = nhl_correlation()

    nba_nfl = pd.merge(nba,nfl, on='city')
    pval_nba_nfl = stats.ttest_rel(nba_nfl['W/L%'],nba_nfl['W-L%'])[1]
    nba_nhl = pd.merge(nba,nhl, on='city')
    pval_nba_nhl = stats.ttest_rel(nba_nhl['W/L%'],nba_nhl['W/L Ratio'])[1]
    mlb_nfl = pd.merge(mlb,nfl, on='city')
    pval_mlb_nfl = stats.ttest_rel(mlb_nfl['W-L%_x'],mlb_nfl['W-L%_y'])[1]
    mlb_nhl = pd.merge(mlb,nhl, on='city')
    pval_mlb_nhl = stats.ttest_rel(mlb_nhl['W-L%'],mlb_nhl['W/L Ratio'])[1]
    mlb_nba = pd.merge(mlb,nba, on='city')
    pval_mlb_nba = stats.ttest_rel(mlb_nba['W-L%'],mlb_nba['W/L%'])[1]
    nhl_nfl = pd.merge(nhl,nfl, on='city')
    pval_nhl_nfl = stats.ttest_rel(nhl_nfl['W/L Ratio'],nhl_nfl['W-L%'])[1]
    
    pv = {'NFL': {"NFL": np.nan, 'NBA': pval_nba_nfl, 'NHL': pval_nhl_nfl, 'MLB': pval_mlb_nfl},
       'NBA': {"NFL": pval_nba_nfl, 'NBA': np.nan, 'NHL': pval_nba_nhl, 'MLB': pval_mlb_nba},
       'NHL': {"NFL": pval_nhl_nfl, 'NBA': pval_nba_nhl, 'NHL': np.nan, 'MLB': pval_mlb_nhl},
       'MLB': {"NFL": pval_mlb_nfl, 'NBA': pval_mlb_nba, 'NHL': pval_mlb_nhl, 'MLB': np.nan}
      }

    p_values = pd.DataFrame(pv)

    return p_values

In [16]:
sports_team_performance()

,NFL,NBA,NHL,MLB
NFL,NaN,0.937445,0.030318,0.803459
NBA,0.937445,NaN,0.022297,0.951629
NHL,0.030318,0.022297,NaN,0.000703
MLB,0.803459,0.951629,0.000703,NaN
